In [ ]:
import os
import numpy as np

# Base directory
base_dir = "/home/jk/Relightable3DGaussian/output_diffuse"
subfolders = ["damwall_harbour", "chapelday_goldenbay", "goldenbay_damwall"]
experiments = [
    "hook150_v3_transl_statictimestep1_r2",
    "jumpingjacks150_v3_tex_statictimestep75_r2",
    "mouse150_v2_transl_statictimestep1_r2",
    "spheres_cube_dataset_v5_statictimestep1_r2",
    "spheres_cube_dataset_v8_spec32_statictimestep1_r2",
    "standup150_v3_statictimestep75_r2"
]

results = {}

for subfolder in subfolders:
    metrics_nvs = {'psnr_nvs': [], 'ssim_nvs': [], 'lpips_nvs': []}
    metrics_pbr_albedo = {
        'psnr_pbr': [], 'ssim_pbr': [], 'lpips_pbr': [],
        'psnr_albedo': [], 'ssim_albedo': [], 'lpips_albedo': []
    }

    for exp in experiments:
        exp_dir = os.path.join(base_dir, subfolder, exp, "neilf")

        # --- Part 1: NVS Metrics ---
        metric_path = os.path.join(exp_dir, "metric.txt")
        if os.path.exists(metric_path):
            with open(metric_path, "r") as f:
                for line in f:
                    key, value = line.strip().split(": ")
                    key_nvs = key + "_nvs"
                    if key_nvs in metrics_nvs:
                        metrics_nvs[key_nvs].append(float(value))

        # --- Part 2: PBR + Albedo Metrics ---
        rli_metric_path = os.path.join(exp_dir, "test_rli/env/metric.txt")
        if os.path.exists(rli_metric_path):
            with open(rli_metric_path, "r") as f:
                for line in f:
                    key, value = line.strip().split(": ")
                    if key in metrics_pbr_albedo:
                        metrics_pbr_albedo[key].append(float(value))

    # Combine all metrics
    combined_means = {}
    for k, v in {**metrics_nvs, **metrics_pbr_albedo}.items():
        combined_means[k] = np.mean(v) if v else None

    results[subfolder] = combined_means

# Print results as table
print(f"{'Subfolder':<25}", end='')
for metric in next(iter(results.values())).keys():
    print(f"{metric:<15}", end='')
print()

for subfolder, metrics in results.items():
    print(f"{subfolder:<25}", end='')
    for value in metrics.values():
        print(f"{value:<15.4f}" if value is not None else f"{'N/A':<15}", end='')
    print()
